In [17]:
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
from typing import *
from scipy.ndimage import gaussian_filter1d
import pandas as pd

In [6]:
behavior_paths = [
    "/data2/kushal/inter_region/JoyJ2/121819_IT/Behavior",
    "/data2/kushal/inter_region/JoyJ2/121819_PT/Behavior",
    "/data2/kushal/inter_region/JoyJ2/122019_IT/Behavior",
    "/data2/kushal/inter_region/JoyJ2/122019_PT/Behavior",
    
    "/data2/kushal/inter_region/JoyM2/072419_IT/Behavior",
    "/data2/kushal/inter_region/JoyM2/072419_PT/Behavior",
    "/data2/kushal/inter_region/JoyM2/072519_IT/Behavior",
    "/data2/kushal/inter_region/JoyM2/072519_PT/Behavior",
    "/data2/kushal/inter_region/JoyM2/072619_IT/Behavior",
    "/data2/kushal/inter_region/JoyM2/072619_PT/Behavior",
]

behavior_paths = [Path(p) for p in behavior_paths]

fname_lift = "lift_frame_num_all_trials.txt"
fname_success = "success_flag_all_trials.txt"

for p in behavior_paths:
    if not p.joinpath(fname_lift).is_file():
        print(f"missing lift indexing: {p}")
    if not p.joinpath(fname_success).is_file():
        print(f"missing success/fail indexing: {p}")

# verify loading the indexing files works
### These lift frame indices are actually garbage and don't make sense, lift frame index is usually ~1600 ¯\\\_(ツ)_/¯ 

In [16]:
np.genfromtxt(p.joinpath(fname_lift), delimiter=",")

array([ 2., 55., 57., 15., 16., 16.,  2., 16., 16., 16.,  3., 20., 55.,
       16., 16., 16., 13., 16., 14.,  1., 16.,  4., 15.,  2., 16., 16.,
       16., 15., 15., 16., 14., 13., 28., 14., 15., 15., 16., 13., 33.,
       16., 15., 27., 16.,  8.,  2., 15., 23.,  7., 11., 15., 16., 15.,
       14., 16., 15., 16., nan,  2., 15.,  6.,  6., 15., 12., 25., 15.,
       12.,  1., 16., 15.,  7., 13., 16., 16., 15., 15.,  6., 15.,  4.])

In [15]:
np.genfromtxt(p.joinpath(fname_success), delimiter=",").astype(bool)

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [22]:
all_trajectories = pd.DataFrame(columns=["traj", "file", "sf"])

for p in tqdm(behavior_paths):
    trk_paths = sorted(p.joinpath("side").glob("trk*v*mat"))
    sfs = np.genfromtxt(p.joinpath(fname_success), delimiter=",").astype(bool)
    
    for tp, sf in zip(trk_paths, sfs):
        traj = loadmat(str(tp))['pred_locs'][:, 0, :]
        
        s = pd.Series(
            {
                "traj": traj,
                "file": tp,
                "sf": sf
            }
        )
        
        # append to dataframe
        all_trajectories.loc[all_trajectories.index.size] = s

100%|█████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.53it/s]


In [25]:
all_trajectories.iloc[0]

traj    [[222.1209111213684, 168.07275390625], [221.94...
file    /data2/kushal/inter_region/JoyJ2/121819_IT/Beh...
sf                                                   True
Name: 0, dtype: object

In [28]:
all_trajectories.to_pickle("./teena_df.pickle")